In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import pandas as pd
from tqdm import tqdm
import time
import pymrmr

from copy import deepcopy
from sklearn.feature_selection import RFECV
from sklearn import svm
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold

In [26]:
data_path = 'data_154.xlsx'
label_path = 'label_154.xlsx'

data = pd.read_excel(data_path, index_col=0, sheet_name=0)
label = pd.read_excel(label_path, index_col=0, sheet_name=0)
features = np.array (list(data.columns))

negative_idx = np.array ((np.arange(22) + 27).tolist() + (np.arange(48) + 106).tolist())

print(data.head(), label.head())

       ak-1_L_thal-mean  ak-1_L_thal-std  ak-1_L_thal-skew  ak-1_L_thal-kurt  \
HT102          0.605042         0.128671         -0.345832          0.060403   
HT103          0.649086         0.133304         -0.238727         -0.237111   
HT105          0.661849         0.170823         -0.002355         -0.595842   
HT106          0.667329         0.128948         -0.183180         -0.034880   
HT107          0.693921         0.155623          0.443730          0.196212   

       ak-1_L_thal-etrp  ak-2_R_thal-mean  ak-2_R_thal-std  ak-2_R_thal-skew  \
HT102          8.750292          0.606492         0.120894         -0.525165   
HT103          8.752230          0.700264         0.114282         -0.033532   
HT105          8.739804          0.653673         0.146263         -0.039633   
HT106          8.754848          0.657183         0.140268         -0.318437   
HT107          8.749220          0.684795         0.139449         -0.211015   

       ak-2_R_thal-kurt  ak-2_R_thal-e

In [27]:
X = data
y = label['T1 Letter Number']

drop = y.notnull()

X = X[drop]
y = y[drop]

YX = pd.concat([y, X], axis=1)
print(YX.head())

       T1 Letter Number  ak-1_L_thal-mean  ak-1_L_thal-std  ak-1_L_thal-skew  \
HT112               1.0          0.667250         0.145615         -0.094611   
HT113               0.0          0.653951         0.155048          0.010153   
HT114               0.3          0.681961         0.168137         -0.194994   
HT115               2.0          0.625602         0.172083         -0.129125   
HT116              -1.7          0.642646         0.159711         -0.107693   

       ak-1_L_thal-kurt  ak-1_L_thal-etrp  ak-2_R_thal-mean  ak-2_R_thal-std  \
HT112         -0.262026          8.749586          0.595198         0.137055   
HT113         -0.465288          8.745306          0.633974         0.147082   
HT114          0.059338          8.742023          0.657136         0.166364   
HT115         -0.544453          8.734418          0.626613         0.153104   
HT116         -0.161909          8.741910          0.680234         0.164955   

       ak-2_R_thal-skew  ak-2_R_thal-k

In [22]:
feature_mrmr = pymrmr.mRMR(YX, 'MIQ', 10)

In [23]:
X_MRMR = X[feature_mrmr]
y = y
print(X_MRMR.head(), y.head())

X_MRMR = X_MRMR.values
y = y.values

       md-R_Pref-kurt  mk-R_Pref-skew  mk-2_R_thal-skew  ak-1_L_thal-kurt  \
HT112       28.288244       -0.664989          0.573990         -0.262026   
HT113       25.865230       -0.525476          0.179915         -0.465288   
HT114        5.269185       -0.256552          0.136331          0.059338   
HT115       24.737511       -0.555679          0.216709         -0.544453   
HT116       23.008476       -0.680608          0.180894         -0.161909   

       mk-R_Pref-mean  awf-CC_Genu_mask-etrp  md-CC_Genu_mask-mean  \
HT112        0.901105               8.998507              0.997993   
HT113        0.901349               9.000073              1.199261   
HT114        0.818448               8.999018              1.319917   
HT115        0.793221               8.995815              1.339361   
HT116        0.872308               9.006571              1.163386   

       awf-2_R_thal-skew  ak-L_Pref-kurt  ak-2_R_thal-kurt  
HT112           0.019168        0.699477          0.334

In [24]:
def greedy_forward_SVM(F, y, num_feature, num_selected, num_repeats, num_test, C_val, Gamma_val, eps_val):
    '''
    Greedy Forward SVM by Dr. Alp
    Args:
        F: data
        y: label
        num_feature: number of feature in total. should be 120 but only 100
        num_selected: number of feature selected
        num_repeats: iter of repeat to avoid noise
        num_test: number of test sample
        C_val: c value as SVM hypter parameter
        Gamma_val: gamma value as SVM hyper parameter
        eps_val: eplison value as SVM hyper parameter
    '''
    best_acc_featsize = np.zeros((num_feature,))  ### accuracy with best subset of different sizes
    all_feat_remained = np.arange(num_feature)
    feat_order = list()

    num_samples = F.shape[0]

    for i in range(num_selected):  ## adds one feature per step
        # iterating through all possible i

        train_acc_cur = np.zeros((len(all_feat_remained), num_repeats))
        test_acc_cur = np.zeros((len(all_feat_remained), num_repeats))
        train_acc_avg = np.zeros((len(all_feat_remained),))
        test_acc_avg = np.zeros((len(all_feat_remained),))
        # print("%d-th feature selection" % ( i+1) )

        for j in range(len(all_feat_remained)):  ## selects one feature out of the remaining ones

            cur_feat_list = deepcopy(feat_order)
            cur_feat_list.append(all_feat_remained[j])
            X = F[:, cur_feat_list]
            # print("%d-th feature selection and feature list= [%s]" % ( i+1, ', '.join(map(str, cur_feat_list))))

            for iter in range(num_repeats):
                # print("%d-th feature selection and %d-th iteration and feature list= [%s]" % ( i+1, iter+1,', '.join(map(str, cur_feat_list))))

                np.random.seed(3 * iter + 10)
                inds = np.random.choice(len(y), num_test)

                X_test = X[inds, :]
                y_test = y[inds]
                X_train = np.delete(X, inds, 0)
                y_train = np.delete(y, inds)

                clf = svm.SVR(C=C_val, epsilon=eps_val, kernel='rbf', degree=4, gamma=Gamma_val, tol=0.001,
                              cache_size=200, max_iter=-1)
                clf.fit(X_train, y_train)
                predicted_train_labels = clf.predict(X_train)
                train_score = clf.score(X_train, y_train)
                # train_error= sum( [1. for k in pred_diff_train if k != 0])/len(predicted_train_labels)
                train_acc_cur[j, iter] = train_score
                predicted_test_labels = clf.predict(X_test)
                test_score = clf.score(X_test, y_test)
                # test_error= sum( [1. for k in pred_diff_test if k != 0])/len(predicted_test_labels)
                test_acc_cur[j, iter] = test_score
                # print("%d-th feature, current list [%s], and its acc= %f" % ( j+1, ', '.join(map(str, cur_feat_list)), test_acc_cur[ j, iter] ))

        for k in range(len(all_feat_remained)):
            train_acc_avg[k] = np.mean(train_acc_cur[k, :])
            test_acc_avg[k] = np.mean(test_acc_cur[k, :])

        best_acc_featsize[i] = np.max(test_acc_avg)
        best_testacc_ind = np.unravel_index(test_acc_avg.argmax(), test_acc_avg.shape)
        feat_order.append(all_feat_remained[best_testacc_ind])
        # print(" current best feature index= %d" %  (all_feat_remained[best_testacc_ind]) )

        all_feat_remained = np.delete(all_feat_remained, best_testacc_ind, 0)

    return feat_order, best_acc_featsize

In [25]:
start_time = time.time()

# 31.62278, 0.31623, 0.00500

num_feature = X_MRMR.shape[1]
num_test = 20
num_repeats = 10  ## shows number of times we shuffle the data and test on testing
train_accuracies = [0] * num_repeats
test_accuracies = [0] * num_repeats

tot_num = X_MRMR.shape[0]

num_selected = 10
eps = 0.005

# C_range = np.logspace(-3, 3, num=5)
# gamma_range = np.logspace(-5, 4, 5)

C_range = np.array([31.62278])
gamma_range = np.array([0.31623])

test_acc_all = np.zeros((C_range.shape[0], gamma_range.shape[0], num_feature))
selected_feat = np.zeros((C_range.shape[0], gamma_range.shape[0], num_selected))

print("Start")
for i in range(len(C_range)):
    for j in range(len(gamma_range)):
        a1, a2 = greedy_forward_SVM(X_MRMR, y, num_feature, num_selected, num_repeats, num_test, C_range[i], gamma_range[j],
                                    eps)
        a3 = np.asarray(a1)
        test_acc_all[i, j, :] = a2
        selected_feat[i, j, :] = a3
        print("(", C_range[i], ",", gamma_range[j], ") value of (C, Gamma), and best feat indices=", a3)

max_ind = np.unravel_index(test_acc_all.argmax(), test_acc_all.shape)
print("maximum test accuracy= %.3f, achieved by using %d features and ( C, Gamma, eps)= ( %.5f, %.5f, %.5f) from 14 metrics" % (
test_acc_all[max_ind], max_ind[2] + 1, C_range[max_ind[0]], gamma_range[max_ind[1]], eps))
print("Selected feature for 3xiter=", selected_feat[max_ind[0], max_ind[1], 0:max_ind[2] + 1])
print(features[np.array(selected_feat[max_ind[0], max_ind[1], 0:max_ind[2] + 1], dtype=np.int)])
# print("\nMaximum Train Accuracy with 100 iteration: %f " % np.max(train_acc_all), "%")
# print("\nMaximum Test Accuracy: %f, by feature indices= %s " % ( np.max(test_acc_avg), subset_indices[best_testacc_ind[0]] ) )

# print("\nall accuracies:", test_acc_avg)

end_time = time.time()
tot_time = end_time - start_time
print("total time=", tot_time)


Start
( 31.62278 , 0.31623 ) value of (C, Gamma), and best feat indices= [3 1 4 5 2 6 7 8 9 0]
maximum test accuracy= -0.073, achieved by using 3 features and ( C, Gamma, eps)= ( 31.62278, 0.31623, 0.00500) from 14 metrics
Selected feature for 3xiter= [3. 1. 4.]
['ak-1_L_thal-kurt' 'ak-1_L_thal-std' 'ak-1_L_thal-etrp']
total time= 1.94856858253479
